<h2><font face=verdona color=yellow><center>Web Extraction Pattern</center></font></h3>
<h4><font face=verdona >
Structure of Extracting Data from websites<br>
⦿ Searching Reputed Used Car Selling Organization Website - Website1_HomeURL, Website2_HomeURL, ...., WebsiteN_HomeURL<br>
⦿ Searching all Cities each organization operates in and fetching links filtering only that city - Website1 : City1_URL,...,CityN_URL<br>
⦿ Fetching links of all cars sold in each city - City1 : Car1_URL, Car2_URL, Car3_URL,........,CarN_URL<br>
⦿ Extracting only required data from each Car link.<br>
</font></h4>
<pre>
                       Website1                             .....                         WebsiteN
                          |                                                                  |
                 -------------------                                                ------------------------
                 |        |         |                                               |                      | 
              City1     City2     City3                                          City1     .....         CityN
                |         |         |                                              |                       |
    -------------      --------     -----------------                  -------------                   --------------
    |     |     |      |      |         |     |     |                  |           |                   |            |                       
  Car1  Car2  Car3    Car1   Car2      Car1  Car2  Car3              Car1  ....  CarN                 Car1  ....  CarN
    ↓     ↓    ↓       ↓      ↓         ↓      ↓    ↓                  ↓          ↓                     ↓           ↓                                     
      Extracted        Extracted           Extracted                     Extracted                        Extracted                                     
        Data             Data                 Data                         Data                              Data                                       


<h4><font face=verdona>DATA CREDITS - Cars24, Carwaale

<H2><font face=georgia color=cyan><center>Importing Modules

In [1]:
import requests
from bs4 import BeautifulSoup
import time 
import numpy as np
import ast
import os
import re
import pandas as pd
from pathlib import Path    

In [2]:
def fetch_and_save(mainpage_link, output_directory):
    try:
        os.makedirs(output_directory, exist_ok=True)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        available_cities = requests.get(mainpage_link, headers=headers)
        available_cities.raise_for_status() 
    except requests.exceptions.RequestException as e:
        print(f"Error fetching main page {mainpage_link}: {e}")
        return {} 

    soup = BeautifulSoup(available_cities.content, 'lxml')
    with open(os.path.join(output_directory, 'main_cities.txt'), 'w', encoding='utf-8')as f:
        f.write(soup.prettify())

In [3]:
mainpage_cars24 = 'https://www.cars24.com/buy-used-cars/'
mainpage_carwale = 'https://www.carwale.com/used/'
mainpage_cardekho = 'https://www.cardekho.com/used-cars/'
mainpage_zigwheels = 'https://www.zigwheels.com/used-car'
mainpage_spinny = 'https://www.spinny.com/used-cars/'

websites = {'cars24' : mainpage_cars24, 'carwale' : mainpage_carwale, 'cardekho' : mainpage_cardekho, 
            'zigwheels': mainpage_zigwheels, 'spinny': mainpage_spinny}

In [4]:
for organization, website_URL in websites.items():
    fetch_and_save(mainpage_link=website_URL, output_directory=organization+'_data')

In [5]:
def extract_citywise_links(root_directory, required_links_extractor, city_extractor, html_tag='a'):
    with open(os.path.join(root_directory, 'main_cities.txt'), 'r', encoding='utf-8')as f:
        content = f.read()
    soup = BeautifulSoup(content, 'lxml')
    href_values = [a.get('href') for a in soup.select(html_tag, href=True)]
    city_links = [required_links_extractor(link) for link in href_values]
    cities = [city_extractor(link) for link in city_links]

    citywise_links = dict()
    for city, link in zip(cities, city_links):
        if city.strip() != '':
            citywise_links[city] = link
    return citywise_links

In [7]:
extract_city_links_cars24 = lambda x: (x if 'buy-used-cars-' in x else '')
city_extractor_cars24 = lambda x: x[len('https://www.cars24.com/buy-used-cars-'):-1]

cars24_citylinks = extract_citywise_links(
    root_directory='cars24_data/', required_links_extractor=extract_city_links_cars24, 
    city_extractor=city_extractor_cars24, html_tag='a.sc-dmfyKz.hgRuOu')

print('Cars24 Available cities and their links \n','-'*30)
cars24_citylinks

Cars24 Available cities and their links 
 ------------------------------


{'delhi-ncr': 'https://www.cars24.com/buy-used-cars-delhi-ncr/',
 'bangalore': 'https://www.cars24.com/buy-used-cars-bangalore/',
 'chennai': 'https://www.cars24.com/buy-used-cars-chennai/',
 'hyderabad': 'https://www.cars24.com/buy-used-cars-hyderabad/',
 'mumbai': 'https://www.cars24.com/buy-used-cars-mumbai/',
 'ahmedabad': 'https://www.cars24.com/buy-used-cars-ahmedabad/',
 'pune': 'https://www.cars24.com/buy-used-cars-pune/',
 'new-delhi': 'https://www.cars24.com/buy-used-cars-new-delhi/',
 'noida': 'https://www.cars24.com/buy-used-cars-noida/',
 'kochi': 'https://www.cars24.com/buy-used-cars-kochi/',
 'ghaziabad': 'https://www.cars24.com/buy-used-cars-ghaziabad/',
 'gurgaon': 'https://www.cars24.com/buy-used-cars-gurgaon/',
 'kolkata': 'https://www.cars24.com/buy-used-cars-kolkata/',
 'lucknow': 'https://www.cars24.com/buy-used-cars-lucknow/',
 'chandigarh-tricity': 'https://www.cars24.com/buy-used-cars-chandigarh-tricity/'}

In [8]:
extract_city_links_carwale = lambda href_val : 'https://www.carwale.com' + href_val
city_extractor_carwale = lambda href_val: href_val[len('https://www.carwale.com/used/'):-1]

carwale_citylinks = extract_citywise_links(root_directory='carwale_data',required_links_extractor=extract_city_links_carwale,
      city_extractor=city_extractor_carwale, html_tag='a.o-aw.o-aE.o-co.o-ml.o-lT.o-mo.o-mO.o-bS.EW3D5f.o-kY.o-f')

print('Carwale Available cities and their links \n','-'*30)
carwale_citylinks

Carwale Available cities and their links 
 ------------------------------


{'mumbai': 'https://www.carwale.com/used/mumbai/',
 'delhi': 'https://www.carwale.com/used/delhi/',
 'bangalore': 'https://www.carwale.com/used/bangalore/',
 'chennai': 'https://www.carwale.com/used/chennai/',
 'hyderabad': 'https://www.carwale.com/used/hyderabad/',
 'kolkata': 'https://www.carwale.com/used/kolkata/',
 'pune': 'https://www.carwale.com/used/pune/',
 'ahmedabad': 'https://www.carwale.com/used/ahmedabad/',
 'chandigarh': 'https://www.carwale.com/used/chandigarh/',
 'coimbatore': 'https://www.carwale.com/used/coimbatore/',
 'lucknow': 'https://www.carwale.com/used/lucknow/',
 'indore': 'https://www.carwale.com/used/indore/'}

In [9]:
extract_city_links_cardekho = lambda x: (x if 'used-cars+in+' in x and '?' not in x else '')
city_extractor_cardekho = lambda x: x[len('https://www.cardekho.com/used-cars+in+'):]

cardekho_citylinks = extract_citywise_links(
    root_directory='cardekho_data', required_links_extractor=extract_city_links_cardekho, 
    city_extractor=city_extractor_cardekho)

print('Cardekho Available cities and their links \n','-'*30)
cardekho_citylinks

Cardekho Available cities and their links 
 ------------------------------


{'ahmedabad': 'https://www.cardekho.com/used-cars+in+ahmedabad',
 'bangalore': 'https://www.cardekho.com/used-cars+in+bangalore',
 'chennai': 'https://www.cardekho.com/used-cars+in+chennai',
 'delhi-ncr': 'https://www.cardekho.com/used-cars+in+delhi-ncr',
 'hyderabad': 'https://www.cardekho.com/used-cars+in+hyderabad',
 'jaipur': 'https://www.cardekho.com/used-cars+in+jaipur',
 'kolkata': 'https://www.cardekho.com/used-cars+in+kolkata',
 'mumbai': 'https://www.cardekho.com/used-cars+in+mumbai',
 'pune': 'https://www.cardekho.com/used-cars+in+pune'}

In [10]:
extract_city_links_zigwheels = lambda x: x
city_extractor_zigwheels = lambda x: x.split('/')[-1].lower()

zigwheels_citylinks = extract_citywise_links(
    root_directory='zigwheels_data', required_links_extractor=extract_city_links_zigwheels, 
    city_extractor=city_extractor_zigwheels, html_tag='a.zw-cmn-blueColor.zw-new-underline')

print('Zigwheels Available cities and their links \n','-'*30)
zigwheels_citylinks

Zigwheels Available cities and their links 
 ------------------------------


{'delhi': 'https://www.zigwheels.com/used-car/Delhi',
 'mumbai': 'https://www.zigwheels.com/used-car/Mumbai',
 'bangalore': 'https://www.zigwheels.com/used-car/Bangalore',
 'pune': 'https://www.zigwheels.com/used-car/Pune',
 'hyderabad': 'https://www.zigwheels.com/used-car/Hyderabad',
 'chennai': 'https://www.zigwheels.com/used-car/Chennai',
 'kolkata': 'https://www.zigwheels.com/used-car/Kolkata',
 'ahmedabad': 'https://www.zigwheels.com/used-car/Ahmedabad',
 'jaipur': 'https://www.zigwheels.com/used-car/Jaipur',
 'coimbatore': 'https://www.zigwheels.com/used-car/Coimbatore',
 'gurgaon': 'https://www.zigwheels.com/used-car/Gurgaon',
 'noida': 'https://www.zigwheels.com/used-car/Noida',
 'thane': 'https://www.zigwheels.com/used-car/Thane',
 'lucknow': 'https://www.zigwheels.com/used-car/Lucknow',
 'chandigarh': 'https://www.zigwheels.com/used-car/Chandigarh',
 'navi-mumbai': 'https://www.zigwheels.com/used-car/Navi-Mumbai'}

In [11]:
extract_city_links_spinny = lambda href_val: ('https://www.spinny.com' + href_val if '/s/' in href_val and 'in' in href_val else '')
city_extractor_spinny = lambda x: x.split('-in-')[-1][:-3]

spinny_citylinks = extract_citywise_links(
    root_directory='spinny_data', required_links_extractor=extract_city_links_spinny, 
    city_extractor=city_extractor_spinny, html_tag='a.styles__navItemTitle')

print('Spinny Available cities and their links \n','-'*30)
spinny_citylinks

Spinny Available cities and their links 
 ------------------------------


{'delhi-ncr': 'https://www.spinny.com/used-cars-in-delhi-ncr/s/',
 'bangalore': 'https://www.spinny.com/used-cars-in-bangalore/s/',
 'hyderabad': 'https://www.spinny.com/used-cars-in-hyderabad/s/',
 'mumbai': 'https://www.spinny.com/used-cars-in-mumbai/s/',
 'pune': 'https://www.spinny.com/used-cars-in-pune/s/',
 'delhi': 'https://www.spinny.com/used-cars-in-delhi/s/',
 'gurgaon': 'https://www.spinny.com/used-cars-in-gurgaon/s/',
 'noida': 'https://www.spinny.com/used-cars-in-noida/s/',
 'ahmedabad': 'https://www.spinny.com/used-cars-in-ahmedabad/s/',
 'chennai': 'https://www.spinny.com/used-cars-in-chennai/s/',
 'kolkata': 'https://www.spinny.com/used-cars-in-kolkata/s/',
 'lucknow': 'https://www.spinny.com/used-cars-in-lucknow/s/',
 'jaipur': 'https://www.spinny.com/used-cars-in-jaipur/s/',
 'chandigarh': 'https://www.spinny.com/used-cars-in-chandigarh/s/',
 'coimbatore': 'https://www.spinny.com/used-cars-in-coimbatore/s/',
 'faridabad': 'https://www.spinny.com/used-cars-in-faridabad

In [16]:
price_range_pairs = {0 : 3, 3 : 10, 10 : 30, 30 : 100}
organizations = ['cars24', 'carwale', 'cardekho', 'zigwheels', 'spinny']
orgs_city_links = [cars24_citylinks, carwale_citylinks, cardekho_citylinks, zigwheels_citylinks, spinny_citylinks]
filter_links = {
    'cars24' : 
    lambda city, new_min, new_max : f'https://www.cars24.com/buy-used-cars-{city}/?f=listingPrice%3Abw%3A{new_min}%2C{new_max}&sort=bestmatch&serveWarrantyCount=true&listingSource=TabFilter&storeCityId=1',
    
    'carwale' : 
    lambda city, new_min, new_max : f'https://www.carwale.com/used/{city}/?kms=0-&year=0-&budget={new_min}-{new_max}&state=-1&so=-1&sc=-1',
    
    'cardekho' : 
    lambda city, new_min, new_max : f'https://www.cardekho.com/used-cars+{new_min}-lakh-to-{new_max}-lakh+in+{city}',
    
    'zigwheels' : 
    lambda city, new_min, new_max : f'https://www.zigwheels.com/used-car/used-cars-between-{new_min}-to-{new_max}-lakhs-in-{city.capitalize()}',
     
    'spinny' : 
    lambda city, new_min, new_max : f'https://www.spinny.com/used-cars-over-{new_min}-lakh-rs-under-{new_max}-lakh-rs-in-{city}/s/'
    }

def extract_cars_data(price_range_pairs, organizations, orgs_city_links, filter_links, output_directory):

    for min, max in price_range_pairs.items():
        for organization, orgwise_links, filter in zip(organizations, orgs_city_links, filter_links.values()):
            filter_urls = []
            os.makedirs(organization+'_data/'+output_directory, exist_ok=True)
            print(organization)
            for city in orgwise_links.keys():
                if 'cars24' == organization:
                    new_min, new_max = min*100000, max*100000
                else :
                    new_min, new_max = min, max

                filter_url = filter(city, new_min, new_max)
                filter_urls.append(filter_url)
            output_file = f"{organization}_data/{output_directory}/cars-{str(min)}-{str(max)}-lakhs-data.txt"
            print(output_file)
            
            with open(output_file, 'w', encoding='utf-8')as f:
                f.write(str(filter_urls))

extract_cars_data(price_range_pairs, organizations, orgs_city_links, filter_links, output_directory='cityprice_data')

cars24
cars24_data/cityprice_data/cars-0-3-lakhs-data.txt
carwale
carwale_data/cityprice_data/cars-0-3-lakhs-data.txt
cardekho
cardekho_data/cityprice_data/cars-0-3-lakhs-data.txt
zigwheels
zigwheels_data/cityprice_data/cars-0-3-lakhs-data.txt
spinny
spinny_data/cityprice_data/cars-0-3-lakhs-data.txt
cars24
cars24_data/cityprice_data/cars-3-10-lakhs-data.txt
carwale
carwale_data/cityprice_data/cars-3-10-lakhs-data.txt
cardekho
cardekho_data/cityprice_data/cars-3-10-lakhs-data.txt
zigwheels
zigwheels_data/cityprice_data/cars-3-10-lakhs-data.txt
spinny
spinny_data/cityprice_data/cars-3-10-lakhs-data.txt
cars24
cars24_data/cityprice_data/cars-10-30-lakhs-data.txt
carwale
carwale_data/cityprice_data/cars-10-30-lakhs-data.txt
cardekho
cardekho_data/cityprice_data/cars-10-30-lakhs-data.txt
zigwheels
zigwheels_data/cityprice_data/cars-10-30-lakhs-data.txt
spinny
spinny_data/cityprice_data/cars-10-30-lakhs-data.txt
cars24
cars24_data/cityprice_data/cars-30-100-lakhs-data.txt
carwale
carwale_da

In [ ]:
def extract_cities_data_from_citylinks(root_directory, citylinks_dict, resume_index=0):
    try : 
        resume = 0
        start_exec = False
        for city, link in citylinks_dict.items():
            if resume == resume_index:
                start_exec = True
            resume +=1
            if start_exec == True:
                time.sleep(5)
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
                content = requests.get(link, headers=headers)
                soup = BeautifulSoup(content.content, 'lxml')
                with open(f'{root_directory}/citywise_cars_data/{city}_cars.txt','w', encoding='utf-8')as f:
                    f.write(soup.prettify())
    except Exception as ex:
        print(ex)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching main page {link}: {e}")
    finally :
        return resume

In [ ]:
resume = extract_cities_data_from_citylinks(root_directory='cars24_data', citylinks_dict=cars24_citylinks)
print('Execution done till index: ',resume, 'out of: ', len(cars24_citylinks))

Execution done till index:  15 out of:  12


In [ ]:
resume = extract_cities_data_from_citylinks(root_directory='carwale_data', citylinks_dict=carwale_citylinks)
print('Execution done till index: ',resume, 'out of: ', len(carwale_citylinks))

Execution done till index:  12 out of:  12


In [68]:
home_url = 'https://www.cars24.com/'

def extract_all_cars_links(citywise_links, root_folder, html_tag, any_processesing):
    try : 
        all_cars_links = list()
        for city in citywise_links.keys():
            print(city)
            file_name = city + '_cars.txt'
            with open(f'{root_folder}/{file_name}','r', encoding='utf-8')as f:
                content = f.read()
                f.close()
            soup = BeautifulSoup(content, 'lxml')
            cars_href_values = [a.get('href') for a in soup.select(html_tag, href=True)]
            correct_cars_links = [any_processesing(x) for x in cars_href_values]
            all_cars_links.extend(cars_href_values)
    except Exception as ex:
        print(ex)
    finally:
        return all_cars_links

In [69]:
print('Cars24\n','-'*30)
no_processing = lambda x: x

all_cars_links_cars24 = extract_all_cars_links(
    citywise_links=cars24_citylinks, root_folder='cars24_data/citywise_cars_data', html_tag='a.styles_carCardWrapper__sXLIp',
    any_processesing=no_processing)

all_cars_links_cars24 = list(set(all_cars_links_cars24))

print('-'*30)
print(f'Extracted all cars links available in the Above cities : ')
print(f'Total links Extracted : ', len(all_cars_links_cars24))

Cars24
 ------------------------------
delhi-ncr
bangalore
chennai
hyderabad
mumbai
ahmedabad
pune
new-delhi
noida
kochi
ghaziabad
gurgaon
kolkata
lucknow
chandigarh-tricity
------------------------------
Extracted all cars links available in the Above cities : 
Total links Extracted :  297


In [70]:
print('Carwale\n','-'*30)
add_prefix_carwale = lambda href_val : 'https://www.carwale.com' + href_val

all_cars_links_carwale = extract_all_cars_links(citywise_links=carwale_citylinks, root_folder='carwale_data/citywise_cars_data', 
                                                html_tag='a.o-js, a.o-C',any_processesing=add_prefix_carwale)

all_cars_links_carwale = list(set(all_cars_links_carwale))

print('-'*30)
print(f'Extracted all cars links available in the Above cities : ')
print(f'Total links Extracted : ', len(all_cars_links_carwale))

Carwale
 ------------------------------
mumbai
delhi
bangalore
chennai
hyderabad
kolkata
pune
ahmedabad
chandigarh
coimbatore
lucknow
indore
------------------------------
Extracted all cars links available in the Above cities : 
Total links Extracted :  90


In [ ]:
def extract_car_data(all_cars_links, start_from_index=0, root_folder='cars24_data/carwise_data', 
                     url_prefix = len("https://www.cars24.com/buy-used-")):
    try:
        for index, car_url in enumerate(all_cars_links[start_from_index:]):
            time.sleep(5.0)
            with open(f'{root_folder}/{car_url[url_prefix:-1]}.txt', 'w', encoding='utf-8')as f:
                content = requests.get(car_url)
                soup = BeautifulSoup(content, 'html.parser')
                f.write(soup.prettify())
    except Exception as ex :
        print(F"Due to exception {ex} stopped at index {start_from_index + index}")
    finally :
        return start_from_index+index
        
resume_index = extract_car_data(all_cars_links)

Due to exception ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)) stopped at index 142


In [ ]:
# resume_index = extract_car_data(all_cars_links, start_from_index=resume_index)
# resume_index

143

In [ ]:
def extract_specs_set(key_classes: str,val_classes: str, file_handling_mode: str, dict_for_empty_data: dict, start_index_finder_func,
        import_data_folder='cars24_data/carwise_data', export_data_folder='cars24_data/required_car_data'):

    for file in os.listdir(import_data_folder):
        with open(os.path.join(import_data_folder, file), 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')

        raw_keys = [tag.get_text(strip=True) for tag in soup.select(key_classes)]
        raw_values = [tag.get_text(strip=True) for tag in soup.select(val_classes)]

        if len(raw_keys) == 0:
            if 'w' in file_handling_mode :
                with open(os.path.join(export_data_folder, file+'_dict.txt'), 'w', encoding='utf-8') as f:
                    f.write(str(dict_for_empty_data))
            else :
                with open(os.path.join(export_data_folder, file+'_dict.txt'), 'r+', encoding='utf-8') as f:
                    dict_data = f.read()
                    dict_data = ast.literal_eval(dict_data)
                    dict_data.update(dict_for_empty_data)
                    f.seek(0)
                    f.truncate()
                    f.write(str(dict_data))
            continue

        try:
            stop_index = raw_keys.index(raw_keys[0], 1) 
        except ValueError:
            stop_index = 11  
        cleaned_keys = raw_keys[:stop_index]

        start_index = start_index_finder_func(raw_values)

        if start_index is None:
            print(f"No matching pattern in: {file}")
            break

        cleaned_values = raw_values[start_index:start_index + len(cleaned_keys)]

        if len(cleaned_keys) == 0 or len(cleaned_values) == 0:
            print(os.path.join(import_data_folder, file))
            break

        clean_data = {key: val for key, val in zip(cleaned_keys, cleaned_values)}
        
        if 'w' in file_handling_mode :
            with open(os.path.join(export_data_folder, file+'_dict.txt'), 'w', encoding='utf-8') as f:
                f.write(str(clean_data))
        else :
            with open(os.path.join(export_data_folder, file+'_dict.txt'), 'r+', encoding='utf-8') as f:
                dict_data = f.read()
                dict_data = ast.literal_eval(dict_data)
                dict_data.update(clean_data)
                f.seek(0)
                f.truncate()
                f.write(str(dict_data))

In [ ]:
def overview_details(raw_values):
    return next(
        (i for i, val in enumerate(raw_values)
            if val[:3].isalpha() and val[3:].strip().isdecimal() and len(val) == 8),
        None
    )
        
extract_specs_set(key_classes='div.styles_itemBody__IMfct p.sc-braxZu.jjIUAi, div.styles_itemBody__IMfct p.sc-kXWJUi.kNWynG', 
                   val_classes='div.styles_itemBody__IMfct p.sc-braxZu.kjFjan, div.styles_itemBody__IMfct p.sc-kXWJUi.gFbJQX',
                   file_handling_mode='w',
                   start_index_finder_func=overview_details,
                   dict_for_empty_data={'Reg. year': None, 'Fuel': None, 'KM driven': None, 'Transmission': None, 'Engine capacity': None, 'Ownership': None, 'Make year': None, 'Spare key': None, 'Reg number': None, 'Insurance': None, 'Insurance type': None},)

In [ ]:
def service_details(raw_values):
    return next(
        (i for i, val in enumerate(raw_values)
        if val[-8:].strip().isalpha() and val[:-8].strip().isdecimal() and 'service' in val),
        None
        )

extract_specs_set(key_classes='div.styles_overviewList__36RdY p.sc-braxZu.lmmumg, div.styles_overviewList__36RdY p.sc-kXWJUi.eAZAjo',
                   val_classes='div.styles_overviewList__36RdY p.sc-braxZu.kjFjan, div.styles_overviewList__36RdY p.sc-kXWJUi.gFbJQX',
                   file_handling_mode='r+',
                   dict_for_empty_data={'Total services': None, 'Last service KM': None, 'Last service date': None},
                    start_index_finder_func=service_details)

In [ ]:



def extract_specs_set2(raw_keys: list, val_classes: str,
                       file_handling_mode: str, dict_for_empty_data: dict,
        import_data_folder='cars24_data/carwise_data', export_data_folder='cars24_data/required_car_data'):

    for import_file, export_file in zip(os.listdir(import_data_folder), os.listdir(export_data_folder)):

        with open(os.path.join(import_data_folder, import_file), 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')

        raw_values = [tag.get_text(strip=True) for tag in soup.select(val_classes)]
        clean_data = {key : val for key, val in zip(raw_keys, raw_values)}

        if len(raw_values) == 0:
            if 'w' in file_handling_mode :
                with open(os.path.join(export_data_folder, export_file), 'w', encoding='utf-8') as f:
                    f.write(str(dict_for_empty_data))
            else :
                with open(os.path.join(export_data_folder, export_file), 'r+', encoding='utf-8') as f:
                    dict_data = f.read()
                    dict_data = ast.literal_eval(dict_data)
                    dict_data.update(dict_for_empty_data)
                    f.seek(0)
                    f.truncate()
                    f.write(str(dict_data))
            continue
        
        if 'w' in file_handling_mode :
            with open(os.path.join(export_data_folder, export_file), 'w', encoding='utf-8') as f:
                f.write(str(clean_data))
        else :
            with open(os.path.join(export_data_folder, export_file), 'r+', encoding='utf-8') as f:
                dict_data = f.read()
                dict_data = ast.literal_eval(dict_data)
                dict_data.update(clean_data)
                f.seek(0)
                f.truncate()
                f.write(str(dict_data))

In [224]:
empty_inspection_details = {'Accident history' : None, 'Odometer tampering':None,
                            'Water damages': None, 'Quality checks': None}

extract_specs_set2(
    list(empty_inspection_details.keys()), 
    'p.sc-braxZu.dDkhIL, p.sc-kXWJUi.jjoOOP',
     'r', empty_inspection_details)

In [228]:
empty_city = {'City' : None}

extract_specs_set2(
    list(empty_city.keys()), 'div.styles_cityName__VUTON p', 'r', empty_city)

In [ ]:
empty_extra_charges = {'Extra_Charges': None}

extract_specs_set2(raw_keys=list(empty_extra_charges.keys()), val_classes='div.extraCharges p.sc-braxZu.lmmumg', 
                   file_handling_mode='r', dict_for_empty_data=empty_extra_charges)

In [220]:
empty_extra_charges = {'Price': None}

extract_specs_set2(raw_keys=list(empty_extra_charges.keys()), val_classes='div.styles_price__3yE9i p.sc-braxZu.hhzsvw', 
                   file_handling_mode='r', dict_for_empty_data=empty_extra_charges)

In [ ]:

tags = ['Displacement (cc)', 'Cylinders', 'Max Power (bhp)', 'Seating Capacity', 'Ground Clearance (mm)', 'Bootspace (litres)']

def extract_specs_set3(tags: list, dict_for_empty_data: dict, 
                               import_data_folder='cars24_data/carwise_data', export_data_folder='cars24_data/required_car_data'):
    
    tags_escaped = [re.escape(tag) for tag in tags]
    pattern = re.compile(rf'\\"label\\":\\"({"|".join(tags_escaped)})\\",\\"value\\":\\"(.*?)\\"')
    
    export_folder_path = Path(export_data_folder)
    import_folder_path = Path(import_data_folder)
    
    for import_file_path, export_file_path in zip(import_folder_path.iterdir(), export_folder_path.iterdir()):
        
        if import_file_path.is_file():
            content = import_file_path.read_text(encoding='utf-8')
            
            clean_data = {tag: None for tag in tags}
            
            for match in pattern.finditer(content):
                label, value = match.groups()
                if label in clean_data:
                    clean_data[label] = value
            
            if not any(clean_data.values()):
                clean_data = dict_for_empty_data
        
        if export_file_path.is_file():
            try:
                dict_data = ast.literal_eval(export_file_path.read_text(encoding='utf-8'))
                dict_data.update(clean_data)
                export_file_path.write_text(str(dict_data), encoding='utf-8')
            except (ValueError, SyntaxError) as e:
                print(f"Error processing {export_file_path}: {e}")


In [ ]:
empty_features_details = {}
for tag in tags:
    empty_features_details[tag] = None

extract_specs_set3(tags,empty_features_details)

In [42]:

def create_df(data_folder='cars24_data/required_car_data'):
    data_folder_path = Path(data_folder)
    data_dicts = []

    for file_path in data_folder_path.iterdir():
        if file_path.is_file():
            content = file_path.read_text()
            data_dict = ast.literal_eval(content)
            data_dicts.append(data_dict)

    return pd.DataFrame.from_records(data_dicts)

df = create_df()
print('Total features :', len(df.columns))
print('Shape of DataFrame:', df.shape)
df.head()

Total features : 27
Shape of DataFrame: (286, 27)


,Reg. year,Fuel,KM driven,Transmission,Engine capacity,Ownership,Make year,Spare key,Reg number,Insurance,...,Water damages,Quality checks,Displacement (cc),Cylinders,Max Power (bhp),Seating Capacity,Ground Clearance (mm),Bootspace (litres),Extra_Charges,Price
0,Jan 2017,Petrol,"31,147 km",Manual,1198cc,2nd,Jan 2017,No,TN21**6350,Jan 2026,...,No water damages,140-quality checks,1198,3,68,5,170,265,"+9,000 other charges",â‚¹2.63 lakh
1,Jun 2017,Petrol,"92,985 km",Manual,1198cc,1st,Apr 2017,No,TS09**1191,Need renewal,...,No water damages,140-quality checks,1198,3,68,5,170,265,"+12,796 other charges",â‚¹2.16 lakh
2,Jul 2018,Petrol,"99,609 km",Manual,1198cc,2nd,Feb 2018,Yes,DL3C**4025,Need renewal,...,No water damages,140-quality checks,1198,3,68,5,170,265,"+13,796 other charges",â‚¹2.15 lakh
3,Jun 2018,Petrol,"20,924 km",Manual,NaN,1st,Apr 2018,No,KL32*7138,Feb 2026,...,No water damages,140-quality checks,None,None,None,None,None,None,"+9,000 other charges",â‚¹2.86 lakh
4,Mar 2019,Petrol,"53,686 km",Manual,1198cc,1st,Jan 2019,No,KL57*5330,Need renewal,...,No water damages,140-quality checks,1198,3,68,5,180,265,"+12,796 other charges",â‚¹3.44 lakh


In [47]:
df

,Reg. year,Fuel,KM driven,Transmission,Engine capacity,Ownership,Make year,Spare key,Reg number,Insurance,...,Water damages,Quality checks,Displacement (cc),Cylinders,Max Power (bhp),Seating Capacity,Ground Clearance (mm),Bootspace (litres),Extra_Charges,Price
0,Jan 2017,Petrol,"31,147 km",Manual,1198cc,2nd,Jan 2017,No,TN21**6350,Jan 2026,...,No water damages,140-quality checks,1198,3,68,5,170,265,"+9,000 other charges",â‚¹2.63 lakh
1,Jun 2017,Petrol,"92,985 km",Manual,1198cc,1st,Apr 2017,No,TS09**1191,Need renewal,...,No water damages,140-quality checks,1198,3,68,5,170,265,"+12,796 other charges",â‚¹2.16 lakh
2,Jul 2018,Petrol,"99,609 km",Manual,1198cc,2nd,Feb 2018,Yes,DL3C**4025,Need renewal,...,No water damages,140-quality checks,1198,3,68,5,170,265,"+13,796 other charges",â‚¹2.15 lakh
3,Jun 2018,Petrol,"20,924 km",Manual,NaN,1st,Apr 2018,No,KL32*7138,Feb 2026,...,No water damages,140-quality checks,None,None,None,None,None,None,"+9,000 other charges",â‚¹2.86 lakh
4,Mar 2019,Petrol,"53,686 km",Manual,1198cc,1st,Jan 2019,No,KL57*5330,Need renewal,...,No water damages,140-quality checks,1198,3,68,5,180,265,"+12,796 other charges",â‚¹3.44 lakh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,Sep 2020,Petrol,"68,987 km",Manual,999cc,2nd,Sep 2020,No,PB10**0309,Dec 2025,...,No water damages,140-quality checks,999,3,76,5,165,280,"+13,000 other charges",â‚¹5.85 lakh
282,Apr 2022,Petrol,"91,574 km",Manual,999cc,1st,Feb 2020,No,TS08**9837,Need renewal,...,No water damages,140-quality checks,999,3,76,5,165,280,None,None
283,Oct 2020,Petrol,"43,812 km",Automatic,999cc,1st,Sep 2020,No,WB12**0909,Need renewal,...,No water damages,140-quality checks,999,3,110,5,165,280,"+68,087 other charges",â‚¹5.8 lakh
284,Jan 2021,Petrol,"28,527 km",Automatic,999cc,1st,Jan 2021,No,UP14**6615,Feb 2026,...,No water damages,140-quality checks,999,3,110,5,165,280,"+10,000 other charges",â‚¹7.95 lakh


In [46]:
df['Price']


0      â‚¹2.63 lakh
1      â‚¹2.16 lakh
2      â‚¹2.15 lakh
3      â‚¹2.86 lakh
4      â‚¹3.44 lakh
           ...     
281    â‚¹5.85 lakh
282            None
283     â‚¹5.8 lakh
284    â‚¹7.95 lakh
285    â‚¹5.08 lakh
Name: Price, Length: 286, dtype: object

Only 286 records so we will have to scrape more data

In [41]:
df

NameError: name 'df' is not defined